# II Parcial: Reconocimiento de Patrones
José Julián Camacho Hernández

## Creación de una red fully-connected

### 1 )

In [408]:
class Perceptron_Multicapa:
    def __init__(self, capas, alpha=0.1, valor_inicial=0.7, omega_lr=0.001):
        self.capas = capas
        self.alpha = alpha
        self.valor_inicial = valor_inicial
        self.omega_lr = omega_lr
        self.bias = []
        self.pesos = []
        self.omegas = []
        # Se define un valor inicial no random para tener consistencia
        # entre diferentes ejecuciones con los mismos parámetros
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos, bias y omegas de cada capa
            peso = np.full((capas[i], capas[i+1]), self.valor_inicial)
            self.pesos.append(peso)
            bias = np.full(capas[i+1], self.valor_inicial)
            self.bias.append(bias)
            omegas = np.full(capas[i+1], self.valor_inicial)
            self.omegas.append(omegas)
        # Se agrega omega solo para la primera capa
        self.omegas.insert(0, np.full(capas[0], self.valor_inicial))
        #print('pesos: ', self.pesos)
        #print('bias: ', self.bias)
        #print('omegas: ', self.omegas)

    # Función de activación prelu con parámetro omega
    def prelu(self, x, omega):
        #print('x', x)
        #print('omega', omega)
        return np.where(x > 0, x, omega)
    
    # Derivada derivada de la función prelu con parámetro omega
    def prelu_derivada(self, x, omega):
        #print('x', x)
        #print('omega', omega)
        return np.where(x > 0, 1, omega)

    # Calcular la salida de cada capa
    def feedforward(self, X):
        capa_activacion = [X]
        #print('capa_activacion: ', len(X))
        for i in range(0, len(self.capas) - 1):
            #print('pesos: ', self.pesos[i].shape)
            #print('capa_activacion[i]: ', capa_activacion[i].shape)
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]

            # Se utiliza la función de activación relu paramétrica
            y = self.prelu(x, self.omegas[i+1])
            capa_activacion.append(y)
        return capa_activacion

    # Calcular el error de la capa de salida
    def backpropagation(self, X, y, capa_activacion):
        error = capa_activacion[-1] - y
        delta = error * self.prelu_derivada(capa_activacion[-1], self.omegas[-1])
        delta_omegas = delta

        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            activacion_anterior = capa_activacion[i-1] if i > 0 else X
            d_peso = np.outer(activacion_actual, delta)
            d_bias = delta
            d_omegas = delta_omegas
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias

            # El omega se va aprendiendo en la red, por el mismo método que los pesos y el bias 
            #print('omegas[i+1]: ', self.omegas[i+1])
            #print('alpha: ', self.alpha)
            #print('d_omegas: ', d_omegas)
            # Omega tiene su propio learning rate
            self.omegas[i+1] -= self.omega_lr * d_omegas
            delta = np.dot(delta, self.pesos[i].T) * self.prelu_derivada(activacion_actual, self.omegas[i])
            delta_omegas = delta

    def entrenar(self, X, y, epochs):
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)
        
            # Imprimir el costo por epoch
            loss = np.mean((capa_activacion[-1] - y)**2)
            print(f"Epoch {epoch}: Costo = {loss}")

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]

### 2 )

In [409]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Cargar el set de datos
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

#========  Feature Engineering  ========#

# Convertir a un problema de clasificación binaria
# y = 0 si target <= 150
# y = 1 si target > 150
y_binary = (y > 150).astype(int)

# Estandarización de los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calcular matriz de covarianza para conocer correlaciones
# cov_matrix = np.cov(X_scaled.T)
# print(cov_matrix)
# Eliminar features relacionados
# 5 y 6 tienen alta covarianza
# X_scaled = np.delete(X_scaled, 5, axis=1)
# Sin embargo, quitar una de las dos no generó mejores resultados

# Dividir en set de entrenamiento y de prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binary, test_size=0.2, random_state=42, stratify=y_binary)

# Print the shapes of the train and test sets
print("Train set - X:", X_train.shape, "y:", y_train.shape)
print("Test set - X:", X_test.shape, "y:", y_test.shape)

Train set - X: (353, 10) y: (353,)
Test set - X: (89, 10) y: (89,)


### 3 )

In [410]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

# Definir parámetros por obtener
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [1500, 2000, 2500]
}

# Crear el modelo y entrenarlo
model = MLPClassifier(hidden_layer_sizes=[10, 32, 16, 4], random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=3)
# Entrenar el modelo
grid_search.fit(X_train, y_train)
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor score:", grid_search.best_score_)


Mejores parámetros: {'alpha': 0.001, 'max_iter': 1500}
Mejor score: 0.7422376744410643


### 4 )

In [411]:
import time

#Tomar tiempo de ejecución
start_time = time.time()

# Instanciar la red neuronal
layers_sizes = [10, 10, 32, 16, 4, 2]
# Se utiliza el alpha recomendado por el gridserach
perceptron = Perceptron_Multicapa(layers_sizes, alpha=0.001, valor_inicial=0.01, omega_lr=0.001)

# Entrenar el modelo
perceptron.entrenar(X_train, np.eye(2)[y_train], epochs=150)

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_test)):
    prediccion = perceptron.predecir(X_test[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

end_time = time.time()
time_taken = end_time - start_time


Epoch 0: Costo = 0.3672495527879206
Epoch 1: Costo = 0.30665274083657695
Epoch 2: Costo = 0.2767881017723403
Epoch 3: Costo = 0.26207720520231376
Epoch 4: Costo = 0.25483411975685355
Epoch 5: Costo = 0.25126916954839185
Epoch 6: Costo = 0.2495149789636838
Epoch 7: Costo = 0.24865191588780614
Epoch 8: Costo = 0.24822729588304998
Epoch 9: Costo = 0.2480183646007178
Epoch 10: Costo = 0.2479155363548153
Epoch 11: Costo = 0.24786490698030608
Epoch 12: Costo = 0.2478399626276678
Epoch 13: Costo = 0.24782766122719507
Epoch 14: Costo = 0.2478215864147774
Epoch 15: Costo = 0.24781858060557935
Epoch 16: Costo = 0.24781708921195522
Epoch 17: Costo = 0.24781634634719082
Epoch 18: Costo = 0.24781597432279995
Epoch 19: Costo = 0.24781578662768367
Epoch 20: Costo = 0.2478156909769103
Epoch 21: Costo = 0.2478156415811647
Epoch 22: Costo = 0.24781561563231033
Epoch 23: Costo = 0.2478156017076853
Epoch 24: Costo = 0.2478155940439233
Epoch 25: Costo = 0.24781558970366332
Epoch 26: Costo = 0.2478155871697

### 5 )

In [412]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

def computeMetrics(y_test, y_pred, time):
    # Calcular la exactitud
    acc = accuracy_score(y_test, y_pred)
    # Calcular F1 score
    f1 = f1_score(y_test, y_pred) 
    # Desplegar métricas
    metrics = {"Accuracy":acc, "F1 Score":f1, "Tiempo de ejecución":time}
    df = pd.DataFrame(metrics, index = [0])
    display(df)

# Calcular la precisión de las predicciones
computeMetrics(y_test, predicciones, time_taken)
print('Resultados esperados: ', y_test[:16])
print('Resultados obtenidos: ', predicciones[:16])

,Accuracy,F1 Score,Tiempo de ejecución
0,0.550562,0.0,9.597318


Resultados esperados:  [1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0]
Resultados obtenidos:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Optimizar la red creada

### 1 )

A partir del análisis de la red en múltiples pruebas, se detectaron los siguientes cambios para mejorar los resultados:

1. Dismunuir la cantidad de capas: se pasa de tener 4 capas ocultas a solo 1.
    
    Al realizar las pruebas con la arquitectura anterior, se notó que los gradientes cuando se tienen tantas capas ocultas, se disparaban hacia valores muy altos representados por infinitos, o muy bajos representados por NaN. Por lo tanto, se decide optar por una sola capa oculta, con el fin de evitar dichos valores extremos.

2. Dismunuir la cantidad de neuronas: se pasa a tener una sola capa oculta de 8 neuronas.

    Al igual que el punto anterior, al realizar las pruebas se notó que aumentar la cantidad de neuronas no implica una mejora en las predicciones del modelo. Por lo tanto, se reduce a tener una única capa de 8 neuronas.

3. Ajustar el valor inicial de los pesos, bias y omega: se pasa de 0.01 a 0.02

    Como se producían valores de infinito o NaN, se ajusta el valor inicial de los pesos, bias y parámetro del pRelun con el fin de intentar obtener una mejor convergencia.
    
4. Disminuir el learning rate de la pRelu: se pasa de 0.001 a 0.00001

    Se notó que al disminuir la tasa de aprendizaje del parámetro de la función de activación pRelu, se logran mejores resultados. Esto puede ser debido a que dicho parámetro en las pruebas anteriores era tan grande que causaba oscilación en el gradiente y no permitía su convergencia. 

5. Dismunuir cantidad de epochs: se pasa de 150 a 50

    Al realizar las pruebas se notó que en este caso específico aumentar las epochs no implicaba una gran mejora en los resultados. Los mejores se obtuvieron con 50 epochs.

### 2 )

In [398]:
#Tomar tiempo de ejecución
start_time = time.time()

# Instanciar la red neuronal
layers_sizes = [10, 8, 2]
perceptron = Perceptron_Multicapa(layers_sizes, alpha=0.001, valor_inicial=0.02, omega_lr=0.00001)

perceptron.entrenar(X_train, np.eye(2)[y_train], epochs=50)

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_test)):
    prediccion = perceptron.predecir(X_test[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

end_time = time.time()
time_taken = end_time - start_time

# Calcular la precisión de las predicciones
computeMetrics(y_test, predicciones, time_taken)
print('Resultados esperados: ', y_test[:16])
print('Resultados obtenidos: ', predicciones[:16])

Epoch 0: Costo = 0.34425977221024584
Epoch 1: Costo = 0.2898697772311098
Epoch 2: Costo = 0.2651656757285754
Epoch 3: Costo = 0.2545208883335004
Epoch 4: Costo = 0.2504325241398088
Epoch 5: Costo = 0.24937397783246623
Epoch 6: Costo = 0.249750047679394
Epoch 7: Costo = 0.2508903856734558
Epoch 8: Costo = 0.252560237631491
Epoch 9: Costo = 0.2546975150368935
Epoch 10: Costo = 0.25731193657317486
Epoch 11: Costo = 0.2603562759872306
Epoch 12: Costo = 0.26371814108147984
Epoch 13: Costo = 0.2671214682998657
Epoch 14: Costo = 0.2702500623739611
Epoch 15: Costo = 0.2728105478343842
Epoch 16: Costo = 0.274655095994448
Epoch 17: Costo = 0.2757633089596598
Epoch 18: Costo = 0.2762538884849358
Epoch 19: Costo = 0.27632434393616245
Epoch 20: Costo = 0.2760441353803243
Epoch 21: Costo = 0.27560759944534563
Epoch 22: Costo = 0.27517522521768933
Epoch 23: Costo = 0.27468247212592456
Epoch 24: Costo = 0.27422071691844135
Epoch 25: Costo = 0.27381237563647404
Epoch 26: Costo = 0.2734620373569423
Epoc

,Accuracy,F1 Score,Tiempo de ejecución
0,0.820225,0.771429,0.857288


Resultados esperados:  [1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0]
Resultados obtenidos:  [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0]


A partir de los cambios realizados, se demuestra una gran mejoría en las métricas de los resultados del modelo.